This is an iPython Notebook designed to help graduate students export their purchases while at UIC into a small, easy to search csv file. Prerequisites to using this script is to download all webpages from ChemPurchase using some tool like '<a href='https://www.downthemall.net/'>DownloadThemAll</a>' etc.

In [3]:
import os
from lxml import html
import csv

## Run a sample

In [4]:
with open('/home/mike/chempurchase/order_detail (1).html', 'r') as f:
    page = f.read()

In [5]:
# read html
tree = html.fromstring(page)

In [6]:
# XPath locations for important data
item_map = {
    'order_no': '/html/body/form/table/tr/td/table[1]/tr[2]/td[1]/div/text()',
    'order_date': '/html/body/form/table/tr/td/table/tr[2]/td[3]/div/text()',
    'vendor': '/html/body/form/table/tr/td/table/tr/td/strong/text()',
    'vendor_address': '/html/body/form/table/tr/td/table/tr/td[2]/text()',
    'part_no_list': '/html/body/form/table/tr/td/table[3]/tr/td[2]/div/text()',
    'part_description_list': '/html/body/form/table/tr/td/table[3]/tr/td[3]/div/text()',
    'part_quantity_list': '/html/body/form/table/tr/td/table[3]/tr/td[4]/div/text()',
    'part_unit_price_list': '/html/body/form/table/tr/td/table[3]/tr/td[5]/div/text()',
    'comment': '/html/body/form/table/tr/td/div/table/tr[2]/table/tr[2]/td[3]/text()'
}


In [9]:
for key in item_map:
    print(
        key,
        tree.xpath(item_map[key])
    )

part_description_list ['ADAPTER ENLARGNG 19/22O 14/20I', 'COND, LIEBG, NO H/UP, 19/22, 19/22', '14/20 DELRIN JOINT CLIP PK10', '19/22 DELRIN JOINT CLIP PK10', 'GLOVE BOX GLOVE SZ10.5 BUTYL 8IN PR2', 'GLOVE DRYBOX BUTYL S10.5 EA=PR']
vendor_address ['1260 Garnet Drive (Make sure Garnet in i', 'Northlake,IL\xa060164', '800-545-8823', 'Status: ']
comment ['Glovebox gloves of both sizes, plus some extra glass adapters, an extra condenser, and joint clips']
part_quantity_list ['3', '1', '1', '1', '1', '1']
order_no ['17793']
vendor ['LabSource Inc.']
part_no_list ['14201576', '14204682', '89059608', '89059610', '82031298', '11000566']
part_unit_price_list ['32.35', '77.65', '45.65', '46.65', '392.45', '368.95']
order_date ['\n                2/23/15              ']


## Run on a directory

In [246]:
out_file = open('/home/mike/chempurchase/csvfile.csv','w')
writer = csv.writer(out_file, delimiter=',')
writer.writerow(
            [
                'Order No.',
                'Order Date',
                'Vendor',
                'Vendor Address',
                'Part No.',
                'Description',
                'Quantity',
                'Unit Price',
                'Comments'
            ]
        )
for root, dirs, files in os.walk("/home/mike/chempurchase/", topdown=False):
    for name in files:
        page = open(os.path.join('/home/mike/chempurchase/', name), 'r')
        if name.endswith('.html'):
            tree = html.fromstring(page.read())
            for j in range(0,len(tree.xpath(item_map['part_no_list']))):
                writer.writerow(
                    [
                        tree.xpath(item_map['order_no'])[0],
                        ''.join(tree.xpath(item_map['order_date'])).replace(" ","").replace("\n",""),
                        tree.xpath(item_map['vendor'])[0],
                        " ,".join(tree.xpath(item_map['vendor_address'])[:-1]),
                        tree.xpath(item_map['part_no_list'])[j],
                        tree.xpath(item_map['part_description_list'])[j],
                        tree.xpath(item_map['part_quantity_list'])[j],
                        tree.xpath(item_map['part_unit_price_list'])[j],
                        tree.xpath(item_map['comment'])[0],
                    ]
                )
out_file.close()